In [1]:
!rm -r DD-GAN-AE
!git clone https://github.com/acse-zrw20/DD-GAN-AE
!pip install ./DD-GAN-AE/
!pip install wandb

rm: cannot remove 'DD-GAN-AE': No such file or directory
Cloning into 'DD-GAN-AE'...
remote: Enumerating objects: 623, done.
remote: Counting objects: 100% (623/623), done.
remote: Compressing objects: 100% (436/436), done.
remote: Total 623 (delta 317), reused 450 (delta 157), pack-reused 0
Receiving objects: 100% (623/623), 49.12 MiB | 8.92 MiB/s, done.
Resolving deltas: 100% (317/317), done.
Processing ./DD-GAN-AE
  Created wheel for DD-GAN-AE: filename=DD_GAN_AE-1.0-cp37-none-any.whl size=27649 sha256=7b7c45cdd600b665c8d14c327d19c5895ea2331e83658ad62623f8383df61248
  Stored in directory: /root/.cache/pip/wheels/4b/73/37/d356abe7b41a5356ad9335d49b8c7680932bbf11cf9ffc6cf1
Successfully built DD-GAN-AE
     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 174kB 49.4MB/s 
     |████████████████████████████████| 143kB 59.5MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created w

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import wandb
import tensorflow as tf
from ddganAE.models import SVDAE
from ddganAE.architectures.svdae import *
from ddganAE.wandb.train_wandb_sf import train_wandb_svdae
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Configuration options for hyperparameter optimization
svdae_sweep_config = {
    "method": "random",
    "metric": {"name": "valid_loss", "goal": "minimize"},
    "parameters": {
        "architecture": {
            "values": [
                "dense",
                "deeper_dense",
                "wider_dense",
                "slimmer_dense",
                "vinicius",
                "smaller_vinicius",
                "slimmer_vinicius",
            ]
        },
        "activation": {"values": ["relu", "elu", "sigmoid", "tanh"]},
        "dense_activation": {"values": ["relu", "linear"]},
        "batch_size": {"values": [32, 64, 128]},
        "learning_rate": {"values": [5e-3, 5e-4, 5e-5]},
        "dropout": {"values": [0.3, 0.55, 0.8]},
        "optimizer": {"values": ["nadam", "adam", "sgd"]},
        "momentum": {"values": [0.8, 0.9, 0.98]},
        "beta_2": {"values": [0.9, 0.999, 0.99999]},
        "batch_normalization": {"values": [True, False]},
        "regularization": {"values": [1e-4, 1e-5, 0]},
    },
}

svdae_sweep_config_opt = {
    "method": "random",
    "metric": {
        "name": "valid_loss",
        "goal": "minimize"
    },
    "parameters": {
        "architecture": {
            "values": [
                "vinicius"
            ]
        },
        "activation": {
            "values": [
                "elu"
            ]
        },
        "dense_activation": {
            "values": [
                "linear"
            ]
        },
        "batch_size": {
            "values": [
                64
            ]
        },
        "learning_rate": {
            "values": [
                0.0005
            ]
        },
        "dropout": {
            "values": [
                0.3
            ]
        },
        "optimizer": {
            "values": [
                "nadam"
            ]
        },
        "momentum": {
            "values": [
                0.8
            ]
        },
        "beta_2": {
            "values": [
                0.999
            ]
        },
        "batch_normalization": {
            "values": [
                False
            ]
        },
        "regularization": {
            "values": [
                0
            ]
        },
        "savemodel": {
            "values": [
                True
            ]
        }
    }
}

svdae_sweep_config['parameters']['datafile'] = {'values': ["/content/drive/MyDrive/Colab Notebooks/data/processed/sf_snapshots_200timesteps_rand.npy"]}
svdae_sweep_config_opt['parameters']['datafile'] = {'values': ["/content/drive/MyDrive/Colab Notebooks/data/processed/sf_snapshots_800timesteps_rand.npy"]}

sweep_id = wandb.sweep(svdae_sweep_config_opt, project='svdae-sf',
                          entity='zeff020')
wandb.agent(sweep_id, train_wandb_svdae, count=1)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 2gjeu2mv
Sweep URL: https://wandb.ai/zeff020/svdae-sf/sweeps/2gjeu2mv


wandb: Agent Starting Run: mlbjm7uy with config:
wandb: 	activation: elu
wandb: 	architecture: vinicius
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	beta_2: 0.999
wandb: 	datafile: /content/drive/MyDrive/Colab Notebooks/data/processed/sf_snapshots_800timesteps_rand.npy
wandb: 	dense_activation: linear
wandb: 	dropout: 0.3
wandb: 	learning_rate: 0.0005
wandb: 	momentum: 0.8
wandb: 	optimizer: nadam
wandb: 	regularization: 0
wandb: 	savemodel: True
wandb: Currently logged in as: zeff020 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/ddganAE/utils.py:49: RuntimeWarning: invalid value encountered in true_divide
  cumulative_info = cumulative_info / cumulative_info[-1]
